In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [56]:
# !pip3 install surprise # 처음 한번만 인스톨

In [57]:
ratings = pd.read_csv("./data/ml-latest-small/ratings.csv")

In [58]:
ratings = ratings.drop('timestamp', axis=1)

## 서프라이즈 패키지용 데이터셋 세팅

In [59]:
from surprise import Reader, Dataset

In [60]:
# 서프라이즈 패키지에게 rating의 최솟값, 최댓값 알려줌
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(ratings, reader)

## train, test 데이터 셋 세팅

In [61]:
from surprise.model_selection import train_test_split

In [62]:
r_train, r_test = train_test_split(data, test_size=0.1, random_state=42)

<br>

<br>

# KNNWithMeans user-user 모델링

### 영화 하나만 예측해보기

In [63]:
from surprise.prediction_algorithms.knns import KNNWithMeans

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [112]:
# knn with means 모델 셋팅
sim_options = {'name': 'cosine',
              'min_support': 1,
              'user_based': True}

model = KNNWithMeans(k=10, min_k=3, sim_options=sim_options)

In [113]:
model.fit(r_train)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [114]:
# test 데이터 실제값 확인 (user_id, item_id, rating)
r_test[:10]

[(325, 3863, 3.0),
 (1, 6, 4.0),
 (104, 4676, 3.0),
 (441, 4402, 5.0),
 (474, 2211, 3.0),
 (298, 8814, 0.5),
 (63, 115617, 5.0),
 (448, 380, 3.0),
 (182, 2022, 5.0),
 (68, 46970, 3.0)]

In [115]:
# 데이터 한 개 뽑아오기 (user_id, item_id, rating)
tr_only = r_test[2]
tr_only

(104, 4676, 3.0)

In [117]:
# 데이터 한개에 대한 모델의 예측값
uid = tr_only[0]
iid = tr_only[1]

tr_only_pred = model.predict(uid, iid) # 예측결과 전체 다 가져오기
tr_only_pred

Prediction(uid=104, iid=4676, r_ui=None, est=1.001341310697164, details={'actual_k': 3, 'was_impossible': False})

In [118]:
tr_only_pred.est # 예측 '값'만 가져오기

1.001341310697164

예측값 결과 설명
- uid : user_id
- iid : item_id
- r_ui : rating(실제값)
- est : 모델이 예측한 값
- actual_k : 지정해준 최소 최대값 사이에서 실제로 이 예측을 하기 위해 사용된 neighbor의 숫자, was_impossible이 True이면 이 정보는 나타나지 않음
- was_impossible : 지정해준 조건 내에서 예측이 가능했는지 불가능했는지 기록된 지표 True일 경우 조건내에서 예측값을 가져오는 것이 불가능했다는 표시
    - reason : 왜 조건 내에서 예측이 불가능했는지 이유를 알려주는 항목

In [125]:
# was_impossible = True인 경우
tr_only = r_test[3]
tr_only

(441, 4402, 5.0)

In [126]:
# 데이터 한개에 대한 모델의 예측값
uid = tr_only[0]
iid = tr_only[1]

tr_only_pred = model.predict(uid, iid) # 예측결과 전체 다 가져오기
tr_only_pred

Prediction(uid=441, iid=4402, r_ui=None, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'})

was_impossible = True일 경우 ratings의 glabal_mean으로 예측.

In [127]:
tr_only_pred.est # 예측 '값'만 가져오기

3.5026225317348376

In [128]:
r_train.global_mean

3.5026225317348376

### r_test 리스트에 있는 영화 모두 예측해보기

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [148]:
sim_options = {'name': 'cosine',
              'min_support': 1,
              'user_based': True}

model = KNNWithMeans(k=10, min_k=3, sim_options=sim_options)

In [149]:
model.fit(r_train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [150]:
predictions = model.test(r_test)
predictions[:10]

[Prediction(uid=325, iid=3863, r_ui=3.0, est=2.7556253072397214, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=1, iid=6, r_ui=4.0, est=4.745895832673508, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=104, iid=4676, r_ui=3.0, est=2.028000598730642, details={'actual_k': 5, 'was_impossible': False}),
 Prediction(uid=441, iid=4402, r_ui=5.0, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid=474, iid=2211, r_ui=3.0, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid=298, iid=8814, r_ui=0.5, est=1.393273476384284, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=63, iid=115617, r_ui=5.0, est=3.738577114356836, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=448, iid=380, r_ui=3.0, est=2.6432329747530563, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=182, iid=20

RMSE 확인해보기

In [151]:
from surprise.accuracy import rmse

In [152]:
rmse(predictions)

RMSE: 0.9110


0.9109803318153976

## KNNWithMeans 모델링 user-user(파라미터 튜닝)

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [144]:
sim_options = {'name': 'cosine',
              'min_support': 1,
              'user_based': True}

model = KNNWithMeans(k=40, min_k=5, sim_options=sim_options, verbose=True)

In [145]:
model.fit(r_train)

Computing the pearson similarity matrix...
Done computing similarity matrix.


RMSE 확인해보기

In [146]:
from surprise.accuracy import rmse

In [147]:
rmse(predictions)

RMSE: 0.9038


0.9038129621238673

<br>

<br>

# KNNWithMeans item-item 모델링

### 영화 하나만 예측해보기

In [153]:
from surprise.prediction_algorithms.knns import KNNWithMeans

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [154]:
# knn with means 모델 셋팅
sim_options = {'name': 'pearson',
              'min_support': 1,
              'user_based': False}

model = KNNWithMeans(k=10, min_k=3, sim_options=sim_options)

In [155]:
model.fit(r_train)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [156]:
# test 데이터 실제값 확인 (user_id, item_id, rating)
r_test[:10]

[(325, 3863, 3.0),
 (1, 6, 4.0),
 (104, 4676, 3.0),
 (441, 4402, 5.0),
 (474, 2211, 3.0),
 (298, 8814, 0.5),
 (63, 115617, 5.0),
 (448, 380, 3.0),
 (182, 2022, 5.0),
 (68, 46970, 3.0)]

In [157]:
# 데이터 한 개 뽑아오기 (user_id, item_id, rating)
tr_only = r_test[2]
tr_only

(104, 4676, 3.0)

In [158]:
# 데이터 한개에 대한 모델의 예측값
uid = tr_only[0]
iid = tr_only[1]

tr_only_pred = model.predict(uid, iid) # 예측결과 전체 다 가져오기
tr_only_pred

Prediction(uid=104, iid=4676, r_ui=None, est=1.7364434288730592, details={'actual_k': 10, 'was_impossible': False})

In [159]:
tr_only_pred.est # 예측 '값'만 가져오기

1.7364434288730592

예측값 결과 설명
- uid : user_id
- iid : item_id
- r_ui : rating(실제값)
- est : 모델이 예측한 값
- actual_k : 지정해준 최소 최대값 사이에서 실제로 이 예측을 하기 위해 사용된 neighbor의 숫자, was_impossible이 True이면 이 정보는 나타나지 않음
- was_impossible : 지정해준 조건 내에서 예측이 가능했는지 불가능했는지 기록된 지표 True일 경우 조건내에서 예측값을 가져오는 것이 불가능했다는 표시
    - reason : 왜 조건 내에서 예측이 불가능했는지 이유를 알려주는 항목

In [160]:
# was_impossible = True인 경우
tr_only = r_test[3]
tr_only

(441, 4402, 5.0)

In [161]:
# 데이터 한개에 대한 모델의 예측값
uid = tr_only[0]
iid = tr_only[1]

tr_only_pred = model.predict(uid, iid) # 예측결과 전체 다 가져오기
tr_only_pred

Prediction(uid=441, iid=4402, r_ui=None, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'})

was_impossible = True일 경우 ratings의 glabal_mean으로 예측.

In [162]:
tr_only_pred.est # 예측 '값'만 가져오기

3.5026225317348376

In [163]:
r_train.global_mean

3.5026225317348376

### r_test 리스트에 있는 영화 모두 예측해보기

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [164]:
sim_options = {'name': 'pearson',
              'min_support': 1,
              'user_based': False}

model = KNNWithMeans(k=10, min_k=3, sim_options=sim_options)

In [165]:
model.fit(r_train)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [166]:
predictions = model.test(r_test)
predictions[:10]

[Prediction(uid=325, iid=3863, r_ui=3.0, est=3.2585027100271002, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=1, iid=6, r_ui=4.0, est=4.611180648089032, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=104, iid=4676, r_ui=3.0, est=1.7364434288730592, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=441, iid=4402, r_ui=5.0, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid=474, iid=2211, r_ui=3.0, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid=298, iid=8814, r_ui=0.5, est=1.0827618349392543, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=63, iid=115617, r_ui=5.0, est=3.8741222791956886, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=448, iid=380, r_ui=3.0, est=3.2599857549857547, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=182, i

RMSE 확인해보기

In [167]:
from surprise.accuracy import rmse

In [168]:
rmse(predictions)

RMSE: 0.9314


0.9314269490018052

## KNNWithMeans 모델링 user-user(파라미터 튜닝)

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [200]:
sim_options = {'name': 'pearson',
              'min_support': 1,
              'user_based': False}

model = KNNWithMeans(k=60, min_k=10, sim_options=sim_options, verbose=True)

In [201]:
model.fit(r_train)

Computing the pearson similarity matrix...
Done computing similarity matrix.


RMSE 확인해보기

In [202]:
from surprise.accuracy import rmse

In [203]:
rmse(predictions)

RMSE: 0.9314


0.9314269490018052

### 그리드서치로 최적 파라미터 확인(기획 확인용)

In [85]:
from surprise.model_selection import GridSearchCV, RandomizedSearchCV

In [143]:
sim_options = {
    "name" : ["cosine"]
    , "min_support" : [1]
    , "user_based" : [True]
}
param_grid = {"k":[10,30,50] , "min_k":[1,5,15,25,30], "sim_options": sim_options}

In [141]:
gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse"], n_jobs=3)

In [142]:
gs.fit(data)

KeyboardInterrupt: 

In [ ]:
gs.best_score["rmse"]

In [ ]:
gs.best_params

In [ ]:
gs.best_params["rmse"]

랜덤서치로 최적 파라미터 확인

In [37]:
from scipy.stats import uniform

In [104]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter distribution using uniform distribution
C = uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [105]:
rs = RandomizedSearchCV(KNNWithMeans, hyperparameters, n_iter=10, measures=[u'rmse'], cv=3, refit=False, return_train_measures=False, n_jobs=1, pre_dispatch=u'2*n_jobs', random_state=42, joblib_verbose=0)

In [106]:
rs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [107]:
rs.best_score["rmse"]

0.9051967113819902

In [108]:
rs.best_params

{'rmse': {'C': 1.49816047538945, 'penalty': 'l1'}}

In [109]:
rs.best_params["rmse"]

{'C': 1.49816047538945, 'penalty': 'l1'}

In [111]:
rs.predict(r_test)

ValueError: refit is False, cannot use predict()